In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import dpsimpy

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09_RX/WSCC-09_RX'
filename = 'WSCC-09_RX_WithoutSyngenParams'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')
download_grid_data(filename+'_DI.xml', url+'_DI.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
final_time = 0.5
time_step = 1e-4

log_dir = "logs"
log_level = dpsimpy.LogLevel.trace

num_copies = 0
num_threads = 0
num_seq = 0
sim_name = "WSCC_9bus_coupled_" + str(num_copies) + "_" + str(num_threads) + "_" + str(num_seq)

def simulate_coupled(filenames, copies, threads, seq, implementation):
    dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name)
    reader = dpsimpy.CIMReader(sim_name, log_level, log_level)
    sys = reader.loadCIM(60, filenames, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.IdealVoltageSource)
    
    # if copies > 0:
    #     multiply_connected(sys, copies, 12.5, 0.16, 1e-6)
    
    sim = dpsimpy.Simulation(sim_name, log_level)
    sim.set_system(sys)
    sim.set_domain(dpsimpy.Domain.DP)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_time_step(time_step)
    sim.set_final_time(final_time)
    sim.set_direct_solver_implementation(implementation)
    sim.run()

In [ ]:
%%capture
simulate_coupled(files, 0, 0, 0, dpsimpy.DirectLinearSolverImpl.SparseLU)
ts_dpsim_lv_sparse = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_sparse = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
%%capture
simulate_coupled(files, 0, 0, 0, dpsimpy.DirectLinearSolverImpl.KLU)
ts_dpsim_lv_klu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_klu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
%%capture
simulate_coupled(files, 0, 0, 0, dpsimpy.DirectLinearSolverImpl.DenseLU)
ts_dpsim_lv_denselu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_denselu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
tolerance = 1e-1

for entry in ts_dpsim_rv_sparse:
    sparse_values = ts_dpsim_rv_sparse[entry].values
    klu_values = ts_dpsim_rv_klu[entry].values
    denselu_values = ts_dpsim_rv_denselu[entry].values
    # 2-norm of errors
    error_1 = np.linalg.norm(sparse_values-klu_values, ord=2)
    error_2 = np.linalg.norm(denselu_values-sparse_values, ord=2)
    assert(error_1 < tolerance)
    assert(error_2 < tolerance)

for entry in ts_dpsim_lv_sparse:
    sparse_values = ts_dpsim_lv_sparse[entry].values
    klu_values = ts_dpsim_lv_klu[entry].values
    denselu_values = ts_dpsim_lv_denselu[entry].values
    # 2-norm of errors
    error_1 = np.linalg.norm(sparse_values-klu_values, ord=2)
    error_2 = np.linalg.norm(denselu_values-sparse_values, ord=2)
    assert(error_1 < tolerance)
    assert(error_2 < tolerance)